In [3]:
#Installing & Loading Packages
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import ranksums

/u/home/c/ctang04/.local/lib/python3.9/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [4]:
# Load the patient methylation profiles
m_log1k_path = "/u/home/c/ctang04/HBV_Code/data/data.log1k.txt"
donors_path = "/u/home/c/ctang04/HBV_Code/data/donors.with.samples.txt"

In [5]:
# Read methylation profile
m_log1k_df = pd.read_csv(m_log1k_path, sep='\t', header=0, index_col=0)
#print(m_log1k_df.columns)

# Read donors file
donors_df = pd.read_csv(donors_path, sep='\t', header=0, quotechar='"')

# Remove duplicate samples by donor
unique_donors_df = donors_df.drop_duplicates(subset='donor')

# Get phase classes from the donors
phases = unique_donors_df['phase_HBV'].unique()
print(phases)

# Define mapping between original phases and desired classes
phase_mapping = {
    "Antiviral Rx": "Antiviral Rx",
    "IAH": "IAH",
    "IT": "IT",
    "RP": "RP",
    "RP and Cirrhosis": "Cirrhosis",
    "Antiviral Rx and Cirrh": "Cirrhosis",
    "SC": "SC",
    "ICP": "ICP",
    "IAH and Cirrhosis": "Cirrhosis",
    "SC and Cirrhosis": "Cirrhosis"
}

# Create a new column in the dataframe with the modified classes
unique_donors_df.loc[:, 'modified_phase'] = unique_donors_df['phase_HBV'].map(phase_mapping)

['Antiviral Rx' 'IAH' 'IT' 'RP' 'RP and Cirrhosis'
 'Antiviral Rx and Cirrh' 'SC' 'ICP' 'IAH and Cirrhosis'
 'SC and Cirrhosis']


/u/local/apps/python/3.9.6/gcc-4.8.5/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [6]:
class_mapping = {
    "IT": "IT",
    "IAH": "IAH",
    "Antiviral Rx": "Other",
    "ICP": "Other",
    "RP": "RP",
    "Cirrhosis": "Other"
}
unique_donors_df.loc[:, 'modified_class'] = unique_donors_df['modified_phase'].map(class_mapping)

In [7]:
# Subset unique_donors_df for the IT class (9)
IT_class_df = unique_donors_df[unique_donors_df['modified_class'] == "IT"]

# Extract sample names
IT_sample_names = IT_class_df['sample'].tolist()

# Subset m_log1k_df based on Active_sample_names
IT_class_data = m_log1k_df.loc[:, IT_sample_names]

print(IT_class_data.shape)   # Displaying the dimensions (rows, columns)

(144560, 9)


In [8]:
# Subset unique_donors_df for the IAH (67)
IAH_class_df = unique_donors_df[unique_donors_df['modified_class'] == "IAH"]

# Extract sample names
IAH_sample_names = IAH_class_df['sample'].tolist()

# Subset m_log1k_df based on Active_sample_names
IAH_class_data = m_log1k_df.loc[:, IAH_sample_names]

print(IAH_class_data.shape)   # Displaying the dimensions (rows, columns)

(144560, 67)


In [9]:
# Subset unique_donors_df for the RP (33)
RP_class_df = unique_donors_df[unique_donors_df['modified_class'] == "RP"]

# Extract sample names
RP_sample_names = RP_class_df['sample'].tolist()

# Subset m_log1k_df based on Active_sample_names
RP_class_data = m_log1k_df.loc[:, RP_sample_names]

print(RP_class_data.shape)   # Displaying the dimensions (rows, columns)

(144560, 33)


In [10]:
# Transpose the DataFrames so that sample names are rows and methylation sites are columns
IT_class_data_t = IT_class_data.T
IAH_class_data_t = IAH_class_data.T
RP_class_data_t = RP_class_data.T

# Add a label column to each transposed DataFrame
IT_class_data_t['label'] = 0  # Label for IT
IAH_class_data_t['label'] = 1  # Label for IAH
RP_class_data_t['label'] = 2  # Label for RP

# Concatenate the transposed DataFrames along the rows (axis=0)
combined_df = pd.concat([IT_class_data_t, IAH_class_data_t, RP_class_data_t])

# Separate features and target
X = combined_df.drop(columns=['label'])
y = combined_df['label']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Print shapes to verify dimensions
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print(y)

X_train shape: (87, 144560)
X_test shape: (22, 144560)
y_train shape: (87,)
y_test shape: (22,)
plasma-646-P9-CH             0
plasma-649-t8-6day-P9-CH     0
plasma-1626-P9-CH            0
plasma-3869-P9-CH            0
plasma-2457-P9-CH            0
                            ..
plasma-3409-P9-CH            2
plasma-2502-P9-CH            2
plasma-2738-P9-CH            2
plasma-2577-5day-P9-CH       2
plasma-2568-r1-4day-P9-CH    2
Name: label, Length: 109, dtype: int64


In [12]:
# Separate the active and inactive groups in the training data
X_train_IT = X_train[y_train == 0]
X_train_IAH = X_train[y_train == 1]
X_train_RP = X_train[y_train == 2]

print(X_train_IT.shape)
print(X_train_IAH.shape)
print(X_train_RP.shape)

(7, 144560)
(54, 144560)
(26, 144560)


In [13]:
#methylation_sites = X_train.columns
#print(methylation_sites.shape)
#print(X_train_active.shape)
#print(X_train_inactive.shape)

# P value and Fold Change

In [14]:
X_train_active_IT = X_train_IT
X_train_inactive_IAH = X_train_IAH

methylation_sites = X_train_IT.columns

p_values_ITvIAH = []
fold_changes_ITvIAH = []

for site in methylation_sites:
    active_values = X_train_active_IT[site].values
    inactive_values = X_train_inactive_IAH[site].values

    # Perform Welch's t-test
    t_stat, p_val = ttest_ind(active_values, inactive_values, equal_var=False)
    p_values_ITvIAH.append(p_val)

    # Calculate fold change
    pseudocount = 0.001
    mean_active = np.mean(active_values)
    mean_inactive = np.mean(inactive_values)
    
    if mean_inactive != 0:
        fold_change = np.log2((mean_active + pseudocount) / (mean_inactive + pseudocount))
    else:
        fold_change = float('NaN')  # Handle division by zero case

    fold_changes_ITvIAH.append(fold_change)

# Create a DataFrame for the results
results_df_ITvIAH = pd.DataFrame({
    'methylation_site': methylation_sites,
    'p_value': p_values_ITvIAH,
    'fold_change': fold_changes_ITvIAH
})

# Filter significant sites based on p-value thresholds
significant_sites05_ITvIAH = results_df_ITvIAH[results_df_ITvIAH['p_value'] < 0.05]
significant_sites05_ITvIAH.to_csv('/u/home/c/ctang04/HBV_Code/output/ITvIAH_p05_ttest_fold_change.csv', index=False)
significant_sites01_ITvIAH = results_df_ITvIAH[results_df_ITvIAH['p_value'] < 0.01]
significant_sites01_ITvIAH.to_csv('/u/home/c/ctang04/HBV_Code/output/ITvIAH_p01_ttest_fold_change.csv', index=False)

# Print the number of significant sites
print(significant_sites05_ITvIAH.shape)
print(significant_sites01_ITvIAH.shape)

(10501, 3)
(4350, 3)


In [15]:
X_train_active_IT = X_train_IT
X_train_inactive_RP = X_train_RP

methylation_sites = X_train_IT.columns

p_values_ITvRP = []
fold_changes_ITvRP = []

for site in methylation_sites:
    active_values = X_train_active_IT[site].values
    inactive_values = X_train_inactive_RP[site].values

    # Perform Welch's t-test
    t_stat, p_val = ttest_ind(active_values, inactive_values, equal_var=False)
    p_values_ITvRP.append(p_val)

    # Calculate fold change with pseudocount
    pseudocount = 0.001
    mean_active = np.mean(active_values)
    mean_inactive = np.mean(inactive_values)
    
    if mean_inactive != 0:
        fold_change = np.log2((mean_active + pseudocount) / (mean_inactive + pseudocount))
    else:
        fold_change = float('NaN')  # Handle division by zero case

    fold_changes_ITvRP.append(fold_change)

# Create a DataFrame for the results
results_df_ITvRP = pd.DataFrame({
    'methylation_site': methylation_sites,
    'p_value': p_values_ITvRP,
    'fold_change': fold_changes_ITvRP
})

# Filter significant sites based on p-value thresholds
significant_sites05_ITvRP = results_df_ITvRP[results_df_ITvRP['p_value'] < 0.05]
significant_sites05_ITvRP.to_csv('/u/home/c/ctang04/HBV_Code/output/ITvRP_p05_ttest_fold_change.csv', index=False)
significant_sites01_ITvRP = results_df_ITvRP[results_df_ITvRP['p_value'] < 0.01]
significant_sites01_ITvRP.to_csv('/u/home/c/ctang04/HBV_Code/output/ITvRP_p01_ttest_fold_change.csv', index=False)

# Print the number of significant sites
print(significant_sites05_ITvRP.shape)
print(significant_sites01_ITvRP.shape)

(7239, 3)
(1901, 3)


In [16]:
X_train_active_RP = X_train_RP
X_train_inactive_IAH = X_train_IAH

methylation_sites = X_train_RP.columns

p_values_RPvIAH = []
fold_changes_RPvIAH = []

for site in methylation_sites:
    active_values = X_train_active_RP[site].values
    inactive_values = X_train_inactive_IAH[site].values

    # Perform Welch's t-test
    t_stat, p_val = ttest_ind(active_values, inactive_values, equal_var=False)
    p_values_RPvIAH.append(p_val)

    # Calculate fold change with pseudocount
    pseudocount = 0.001
    mean_active = np.mean(active_values)
    mean_inactive = np.mean(inactive_values)
    
    if mean_inactive != 0:
        fold_change = np.log2((mean_active + pseudocount) / (mean_inactive + pseudocount))
    else:
        fold_change = float('NaN')  # Handle division by zero case

    fold_changes_RPvIAH.append(fold_change)

# Create a DataFrame for the results
results_df_RPvIAH = pd.DataFrame({
    'methylation_site': methylation_sites,
    'p_value': p_values_RPvIAH,
    'fold_change': fold_changes_RPvIAH
})

# Filter significant sites based on p-value thresholds
significant_sites05_RPvIAH = results_df_RPvIAH[results_df_RPvIAH['p_value'] < 0.05]
significant_sites05_RPvIAH.to_csv('/u/home/c/ctang04/HBV_Code/output/RPvIAH_p05_ttest_fold_change.csv', index=False)
significant_sites01_RPvIAH = results_df_RPvIAH[results_df_RPvIAH['p_value'] < 0.01]
significant_sites01_RPvIAH.to_csv('/u/home/c/ctang04/HBV_Code/output/RPvIAH_p01_ttest_fold_change.csv', index=False)

# Print the number of significant sites
print(significant_sites05_RPvIAH.shape)
print(significant_sites01_RPvIAH.shape)

(5138, 3)
(823, 3)


# Union

In [17]:
union_significant_sites05_1vRest = pd.concat([significant_sites05_ITvRP,significant_sites05_ITvIAH,significant_sites05_RPvIAH])
union_significant_sites05_1vRest.drop_duplicates().reset_index(drop=True)
print(union_significant_sites05_1vRest)

union_significant_sites01_1vRest = pd.concat([significant_sites01_ITvRP,significant_sites01_ITvIAH,significant_sites01_RPvIAH])
union_significant_sites01_1vRest.drop_duplicates().reset_index(drop=True)
print(union_significant_sites01_1vRest)

                 methylation_site   p_value  fold_change
16      chr10_100992139_100992258  0.007114    -9.256017
17      chr10_100992275_100992394  0.017237    -9.008365
18      chr10_100992374_100992493  0.017165    -8.966021
40      chr10_101089800_101089919  0.027449    -8.820565
41      chr10_101089922_101090041  0.002009    -9.626346
...                           ...       ...          ...
121216     chr6_90120915_90121034  0.044997    -6.636230
121515   chr7_132260465_132260584  0.025725     2.528339
122693   chr8_145911314_145911433  0.047533          NaN
122720     chr8_17658570_17658689  0.024195          NaN
123445   chr9_132199788_132199907  0.046934          NaN

[22878 rows x 3 columns]
                 methylation_site   p_value  fold_change
16      chr10_100992139_100992258  0.007114    -9.256017
41      chr10_101089922_101090041  0.002009    -9.626346
59      chr10_101287567_101287686  0.001957     0.583280
80      chr10_101294629_101294748  0.004394    -9.304916
111  

In [18]:
fold_change_threshold = 2
significant_sites_05_fold_change = union_significant_sites05_1vRest[union_significant_sites05_1vRest['fold_change'] > fold_change_threshold]
print(significant_sites_05_fold_change)

                 methylation_site   p_value  fold_change
10941   chr12_108238515_108238634  0.003872     2.071044
16192     chr13_20806709_20806828  0.032935     2.226247
16487     chr13_28368172_28368291  0.025739     2.391563
16720     chr13_30982804_30982923  0.031446     2.220010
23433     chr15_81073003_81073122  0.024460     2.403410
...                           ...       ...          ...
116112   chr2_219867615_219867734  0.038477     3.688468
118249     chr3_98451598_98451717  0.018071     4.225913
118250     chr3_98451691_98451810  0.005150     4.788858
119636     chr5_31855140_31855259  0.002411     4.677501
121515   chr7_132260465_132260584  0.025725     2.528339

[1061 rows x 3 columns]


In [19]:
significant_sites_05 = significant_sites_05_fold_change['methylation_site'].tolist()

# Select significant features from the training and testing data
X_train_significant = X_train[significant_sites_05]
X_test_significant = X_test[significant_sites_05]


print(X_train_significant.shape)
print(X_test_significant.shape)
print(y_train.shape)
print(y_test.shape)


(87, 1061)
(22, 1061)
(87,)
(22,)


In [28]:
from sklearn.preprocessing import label_binarize
rf_model = RandomForestClassifier(
    max_depth=None,
    max_features='log2',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=200
)

# Fit the model on the training data
rf_model.fit(X_train_significant, y_train)

# Predict on the test data
y_pred = rf_model.predict(X_test_significant)

# Predict probabilities for AUC calculation
y_pred_prob = rf_model.predict_proba(X_test_significant)

# Check if the classification is binary or multiclass
if len(set(y_train)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_prob[:, 1])
else:  # Multiclass classification
    y_test_bin = label_binarize(y_test, classes=list(set(y_train)))
    auc = roc_auc_score(y_test_bin, y_pred_prob, multi_class='ovo')

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)
print(f"AUC-ROC: {auc}")

Accuracy: 0.6363636363636364
Confusion Matrix:
[[ 0  2  0]
 [ 0 12  1]
 [ 0  5  2]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.63      0.92      0.75        13
           2       0.67      0.29      0.40         7

    accuracy                           0.64        22
   macro avg       0.43      0.40      0.38        22
weighted avg       0.59      0.64      0.57        22

AUC-ROC: 0.5558099308099308


/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Define the scorer for AUC
auc_scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')

# Set up Grid Search with cross-validation
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           scoring=auc_scorer,
                           cv=5,  # 5-fold cross-validation
                           n_jobs=-1,  # Use all available cores
                           verbose=0)

# Fit the Grid Search to the training data
grid_search.fit(X_train_significant, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print(f"Best parameters: {best_params}")

# Predict on the test data
y_pred = best_rf_model.predict(X_test_significant)
y_pred_prob = best_rf_model.predict_proba(X_test_significant)

# Calculate AUC
if len(set(y_train)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_prob[:, 1])
else:  # Multiclass classification
    y_test_bin = label_binarize(y_test, classes=list(set(y_train)))
    auc = roc_auc_score(y_test_bin, y_pred_prob, multi_class='ovo')

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Calculate sensitivity (recall) for each class
sensitivity = np.diag(cm) / np.sum(cm, axis=1)

# Calculate specificity for each class
specificity = []

for i in range(cm.shape[0]):
    # True negatives (TN) for class i
    TN = np.sum(cm) - np.sum(cm[i, :]) - np.sum(cm[:, i]) + cm[i, i]
    # False positives (FP) and true negatives (TN) for class i
    FP_plus_TN = np.sum(cm) - np.sum(cm[:, i])
    # Specificity for class i
    if FP_plus_TN == 0:
        specificity_i = 0
    else:
        specificity_i = TN / FP_plus_TN
    specificity.append(specificity_i)

# Print results for sensitivity and specificity
for i in range(cm.shape[0]):
    print(f'Class {i}: Sensitivity (Recall) = {sensitivity[i]:.2f}, Specificity = {specificity[i]:.2f}')

# Print evaluation results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"ROC AUC: {auc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
319 fits failed with the following error:
Traceback (most recent call last):
  File "/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/u/home/c/ctang04/.local/lib/python3.9/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File 

Best parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
[[ 0  2  0]
 [ 0 12  1]
 [ 0  5  2]]
Class 0: Sensitivity (Recall) = 0.00, Specificity = 0.91
Class 1: Sensitivity (Recall) = 0.92, Specificity = 0.67
Class 2: Sensitivity (Recall) = 0.29, Specificity = 0.74
Accuracy: 0.6363636363636364
Confusion Matrix:
[[ 0  2  0]
 [ 0 12  1]
 [ 0  5  2]]
ROC AUC: 0.5705331705331705
Sensitivity: [0.         0.92307692 0.28571429]
Specificity: [0.9090909090909091, 0.6666666666666666, 0.7368421052631579]
